<a href="https://colab.research.google.com/github/ElisabethShah/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Logistic%20Regression%20Assignment%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression: Water Pumps in Tanzania

#### Objectives
- begin with baselines for classification
- use classification metric: accuracy
- do train/validate/test split
- use scikit-learn for logistic regression
- submit to predictive modeling competitions

## Install and import libraries

In [2]:
!pip install pandas-profiling

In [3]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.4MB/s 


In [0]:
import category_encoders as ce
import pandas as pd
import pandas_profiling

## Import data

In [0]:
LOCAL = '../data/tanzania/'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train_features = pd.read_csv(WEB + 'train_features.csv')
train_labels = pd.read_csv(WEB + 'train_labels.csv')
test_features = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

## Do train/validate/test split

## Begin with baselines for classification